This notebook demonstrates new features in the develop branch of vrs. The primary new features are Repeats and Abundance, and secondary classes that enable their use in representing common variation that involves the duplication of sequence.

The corresponding schema (currently) exists only in the vrs develop branch. It also requires an up-to-date vrs-python package.

**Models in this notebook are under development. They are likely to change before release and may not ever be released.  Don't use them yet.** 

In [1]:
from ga4gh.core import ga4gh_identify, ga4gh_serialize
from ga4gh.vrs import models, vrs_enref, vrs_deref
from nbsupport import ppo, translate_sequence_identifier

In [2]:
from ga4gh.vrs._internal.models import _load_vrs_models
models = _load_vrs_models()

# New Features

## NestedInterval

**Summary:** NestedInterval is a new Interval type represent regions where the start and end coordinates are uncertain but may be bounded. This class is not identifiable.

In [3]:
ni = models.NestedInterval(
    inner = models.SimpleInterval(start=20, end=30),
    outer = models.SimpleInterval(start=10, end=40))
ppo(ni)

{
  "inner": {
    "end": 30,
    "start": 20,
    "type": "SimpleInterval"
  },
  "outer": {
    "end": 40,
    "start": 10,
    "type": "SimpleInterval"
  },
  "type": "NestedInterval"
}


In [4]:
sl = models.SequenceLocation(
    sequence_id = "ga4gh:SQ.0123abcd",
    interval = ni)
ppo(sl)

{
  "interval": {
    "inner": {
      "end": 30,
      "start": 20,
      "type": "SimpleInterval"
    },
    "outer": {
      "end": 40,
      "start": 10,
      "type": "SimpleInterval"
    },
    "type": "NestedInterval"
  },
  "sequence_id": "ga4gh:SQ.0123abcd",
  "type": "SequenceLocation"
}


**Commentary**

NestedIntervals are modeled as a pair of `inner` and `outer` ranges, each of which is a SimpleInterval.  A `start` and `end` are consistent with a NestedInterval if they meet these criteria: `outer.start <= start <= inner.start <= inner.end <= end <= outer.end`.

NestedIntervals are conceptualy related to, and interchangeable with, intervals of intervals (i.e., start and end, each of which is defined by an interval).

Converting HGVS expressions to NestedIntervals requires care. Imagine this sequence with an exon (or any feature) at base positions 4-6 inclusive and sequence DEF.

```
base:     1   2   3   4   5   6   7   8   9
          a   b   c [ D   E   F ] g   h   i
i-base:  0  1   2   3   4   5   6   7   8   9

```

HGVS for the region to the left and right (abc, ghi) would be `(1_3)_(7_9)`, corresponding to interbase ranges (0,3) and (6,9). As a nested interval, the range becomes inner=(3,6) and outer=(0,9).

## Gene

**Summary:** The Gene class wraps an external definition of a gene to be used as the subject of a abundance statement. For example, "PTEN del". Gene is not an identifiable class.


Genes are intended to be used to represent systemic variation, not molecular variation.

In [5]:
g = models.Gene(gene_id="ncbigene:1234")
ppo(g)

{
  "gene_id": "ncbigene:1234",
  "type": "Gene"
}


## Sequence Expressions (Not Identfiable)

Sequence Expressions provide mechanisms to describe the sequence that exists at a particular location. 

The most primitive ways to describe such sequence is through `LiteralSequence`, which specifies the sequence directly in the model, and `DerivedSequence`, which specifies sequence with respect to a SequenceLocation. A `DerivedSequence` may also specify transformations, such as `reverse`, `complement`, and `reversecomplement`. Future extensions may provide metods to describe approximate sequence matching (other than by IUPAC ambiguity codes), sequences constrained by size or size multiples (e.g., modulo 3).

### LiteralSequence

In [7]:
ls = models.LiteralSequenceExpression(sequence="ACGT")
ppo(ls)

{
  "sequence": "ACGT",
  "type": "LiteralSequenceExpression"
}


### DerivedSequence

**Summary:** A DerivedSeqeuence is a sequence derived from a location, optionally with transformation.

In [8]:
sl = models.SequenceLocation(
        sequence_id="ga4gh:SQ.0123abcd",
        interval=models.SimpleInterval(start=22, end=33))
ppo(sl)

{
  "interval": {
    "end": 33,
    "start": 22,
    "type": "SimpleInterval"
  },
  "sequence_id": "ga4gh:SQ.0123abcd",
  "type": "SequenceLocation"
}


In [9]:
ds = models.DerivedSequenceExpression(location=sl)
ppo(ds)

{
  "location": {
    "interval": {
      "end": 33,
      "start": 22,
      "type": "SimpleInterval"
    },
    "sequence_id": "ga4gh:SQ.0123abcd",
    "type": "SequenceLocation"
  },
  "type": "DerivedSequenceExpression"
}


In [11]:
ds = models.DerivedSequenceExpression(location=sl)
ppo(ds)

{
  "location": {
    "interval": {
      "end": 33,
      "start": 22,
      "type": "SimpleInterval"
    },
    "sequence_id": "ga4gh:SQ.0123abcd",
    "type": "SequenceLocation"
  },
  "type": "DerivedSequenceExpression"
}


### RepeatedSequence

**Summary:** A RepeatSequence is a contiguous span of tandem copies of a sequence of any length (length >= 1). The number of copies may be known precisely or may be specified as a range (min, max). A RepeatSequence is primarily used within an Allele to create a repeat within a reference sequence.

In [15]:
rs = models.RepeatedSequenceExpression(
    seq_expr=models.LiteralSequenceExpression(sequence="CAG"),
    count={"min": 5, "max": 10})
ppo(rs)

{
  "count": {
    "max": 10,
    "min": 5,
    "type": "CopyCount"
  },
  "seq_expr": {
    "sequence": "CAG",
    "type": "LiteralSequenceExpression"
  },
  "type": "RepeatedSequenceExpression"
}


In [17]:
sl = models.SequenceLocation(
    sequence_id="ga4gh:SQ.abc123",
    interval=models.SimpleInterval(start=20, end=30))
rs = models.RepeatedSequenceExpression(
    seq_expr=models.DerivedSequenceExpression(location=sl),
    count={"min": 5, "max": 10})
ppo(rs)

{
  "count": {
    "max": 10,
    "min": 5,
    "type": "CopyCount"
  },
  "seq_expr": {
    "location": {
      "interval": {
        "end": 30,
        "start": 20,
        "type": "SimpleInterval"
      },
      "sequence_id": "ga4gh:SQ.abc123",
      "type": "SequenceLocation"
    },
    "type": "DerivedSequenceExpression"
  },
  "type": "RepeatedSequenceExpression"
}


In [21]:
cnv = models.CopyNumber(
    subject=models.Gene(gene_id="ncbigene:672"),
    copies=models.CopyCount(min=0, max=5, absolute_measure=True))
ppo(cnv)

{
  "copies": {
    "absolute_measure": true,
    "max": 5,
    "min": 0,
    "type": "CopyCount"
  },
  "subject": {
    "gene_id": "ncbigene:672",
    "type": "Gene"
  },
  "type": "AbsoluteAbundance"
}


---
# Specific Examples

The examples below use variation from ClinVar and VICC to demonstrate the above classes.

## VCV000528890.1 NC_000001.11:g.(?\_218346682)\_(218441382\_?)del


Three possible interpretations:
* Molecular variation w/del
* MV w/Repeat state 0
* Abundance abs copy = 1

In [ ]:
a = models.Allele(
    location=models.SequenceLocation(
        sequence_id=translate_sequence_identifier("NC_000001.11"),
        interval=models.NestedInterval(
            inner=models.SimpleInterval(start=218346681, end=218441382),
            outer=models.SimpleInterval(start=None, end=None))),
    state=models.SequenceState(sequence=''))
ppo(a)

## VCV000665644.1 NC_000001.10:g.(?\_15764951)\_(15765010\_?)dup

In [ ]:
# if the copies are not known to be tandem, this should be an Abundance statement
a = models.Abundance(
    subject=models.SequenceLocation(
        sequence_id=translate_sequence_identifier("NC_000001.11"),
        interval=models.NestedInterval(
            inner=models.SimpleInterval(start=15764950, end=15765010),
            outer=models.SimpleInterval(start=None, end=None))),
    amount={"min":2, "max": 2, "measure": "AbsoluteCount"})
a.as_dict()

## VCV000662440.1 NM_003000.2(SDHB):c.656_707dup (p.Pro237_Phe238insAspTer)

In [ ]:
a = models.Allele(
    location=models.SequenceLocation(
        sequence_id=translate_sequence_identifier("NM_003000.2"),
        interval=models.SimpleInterval(start=655, end=707)),
    state=models.RepeatState(copies={"min":2, "max": 2}))
a.as_dict()

## VCV000007621.2 NM_000214.3(JAG1):c.2091_2095GAAAG[1] (p.Gly699fs)
```
This variant is incorrectly written.  The repeat unit GAAAG should be left-shuffled to GGAAA, making the left-shifted version c.2090_2094GGAAA.

 c.2081     2091     2101     2111
 n.2551     2561     2571     2581  
      |        |        |        |
      AAATGGGTGGAAAGGAAAGACCTGCCAC

Converting a repeat to a SequenceState requires examining downstream sequence for the repeat unit, and inserting or deleting as necessary.  Imagine that this sequence had five GAAAG repeats and we got `[1]`; then, 4 repeats would be deleted, which is evident only from sequence context.

See note below about sequence locations for repeats.
```

### Discuss: Location of repeats

```
 n.2551              2561              2571              2581  
 c.2081              2091              2101              2111
      |                 |                 |                 |
      A A A T G G G T G G A A A G G A A A G A C C T G C C A C
                      [-------] [-------] - 

* What is the assertion of reference? Two GGAAA at 2090?
* The variant above is an assertion of only 1 repeat. Can we assert zero repeats?
* Must be left aligned? How does that comport with fully justified?

```

In [ ]:
a = models.Allele(
    location=models.SequenceLocation(
        sequence_id=translate_sequence_identifier("NM_000214.3"),
        interval=models.SimpleInterval(start=2089, end=2089)),
    state=models.RepeatState(sequence="GGAAA", copies={"min": 1,"max": 1}))
a.as_dict()

## VCV000144802.1 GRCh38/hg38 1p36.32(chr1:2651321-2701929)x1
⇒ make it possible to represent cytoband and coordinate representations, but not both in one object


In [ ]:
cl = models.ChromosomeLocation(chr="1",
    interval=models.CytobandInterval(start="p36.32", end="p36.32"))
cl.as_dict()

In [ ]:
ab = models.Abundance(
    location=cl,
    amount={
        "min": 1, "max": 1})
ab.as_dict()

In [ ]:
ga4gh_identify(ab)

## VCV000149842.1 GRCh38/hg38 15q11.2(chr15:25334870-25351819)x3
⇒ specified with HGVS NC_000015.8:g.(?_23131110)_(23148059_?)dup, ...

## VCV000395246.1 GRCh37/hg19 Xq21.33-28(chrX:94043221-155246585)x1

## VCV000394192.1 GRCh37/hg19 Xp11.4(chrX:40456453-40487150)x2

## 3 copies EGFR

Larry:
ab = models.Abundance(
    molstate = Gene('ncbigene:1234')
    amount={min: 3, max: 3}
    )

Alex:
ab = models.Abundance(
    molvar = Allele(
       location = Gene('ncbigene:1234'),
       state = AmbiguousState()),
    amount={min: 3, max: 3}
    )

ab = models.Abundance(
    molvar = Allele(
       location = NM_1234.4:22_33,
       state = sequence='' ),
    amount={min: 3, max: 3}
    )



ab = models.Abundance(
    molvar = Allele(
       location = NM_01234.5,
       state = AmbiguousState()),
    amount={min: 3, max: 3}
    )

ab = models.Abundance(
    molvar = Transcript(NM_01234.5),
    amount={min: 3, max: 3}
    )

## EID473 ... “increased copy number or amplification of EGFR”... (>8x copies)

## EID5925 “increased EGFR gene copy number”

## VCV000254074.1 NM_007294.3(BRCA1):c.5075-?_5277+?dup203

## VCV000145395.1 GRCh38/hg38 1p36.33(chr1:844353-911241)x3 (FAM41C , LINC01128)

## VCV000395687.1 GRCh37/hg19 Xp22.33-q28(chrX:70297-155255792)
NCBI calls this a "copy number loss" with corresponding HGVS `NC_000023.10:g.(?_70297)_(155255792_?)del`

# Discussion/Questions/Decisions

## What is the location of a repeat?

A repeat should be located over the entire region of the repeated sequence. Rationale: RepeatState is essentially a delins, where the ins is a repeated sequence.

## Should we permit copy number zero?

RepeatState and Abundance may specify zero repeats/counts. 

Rationale: Users may wish to express count ranges that include 0, such as 0 <= count <= 2. Using a distinct deletion state in lieu of 0 would create significant discontinuities in the data model.

## How to describe repeat and abundance source sequences ?

Obvious needs: 1) inline? sequence, 2) sequence from SequenceLocation, 3) arbitrary sequence constrained by size (e.g., modulo 3). 

Re: "Approximate" repeats: We have yet to come up with a precise definition of approximate, that is a characterization of the ways in which a sequence might be approximate.

Reece thinks that it's important that an expression can be matched against a query sequence. Unclear whether other think that this is important too. A related but distinct issue is whether an expression can generate a sequence (as with a grammar) and whether that's finite.


## Are RepeatState instances normalized?

RepeatState instances are not normalized. Rationale: Data producers may choose a repeat unit that may have meaning, such as starting on a codon boundary. However, in the absence of a reason to choose a particular repeat unit, users should use the left-most representation of the repeat accounting for circular permutations of the repeat unit (as with normalization).


## What about digest?

In VRS, the digest is always a digest of the data structure.  If the implied sequence is well-defined, it the repeat must be converted to an equivalent SequenceState Allele. 


##  Can a SequenceLocation be used as a Sequence?


## Should VRS require all properties explicitly?

## Nerding about Sequence State languages

Sequence States might be considered to provide two related but distinct functions: *generating* sequence and *matching* sequence. Generating is the process of emiting the sequence or sequences that might match a given sequence state. Matching is the process of determining whether a given sequence satisfies the current state.

For literal sequences without ambiguity codes, the generated sequence is the sequence that defines the sequence state. For an Allele with such a sequence state, we could infer the resulting sequence. Or, given a sequence, we could easily assess whether it matches.

For sequences with ambiguity codes, this becomes harder. For example, 'ANT' as a DNA sequences would match 4 DNA sequences and would generate 4 DNA sequences. Allowing ambiguity codes means that it is impossible to infer a unique sequence from a given Allele. Furthermore, combinatorial expansion grows quickly: ANNT would generate and match 4^2 = 16 sequences.  Despite the combinatorial expansion, these combinations could be enumerated.

More expressive Sequence States, which might be supported in the future, create a language that is no longer enumerable at all, but could still be matched. For example, Sequence States that support regular expressions (AN\*T) or unbounded sizes ("at least 5 CAG repeats") create infinite sets which can no longer be completely enumerated. However, a given sequence may still be matched to such an expression.

Expressions fall into a few categories:
* Countable, Unary -- expressions that generate only one sequence. Examples: Literal, Inferred, Inverted (w/no ambiguity codes)
* Countable, non-unary -- expressions that generate more than one sequence. Examples: Literal, regexp (w/ambiguity codes)
* Uncountable -- expressions generate an infinite number of sequences (e.g., "size > 5 nt").

This is relevant to VRS because we *may* want to describe or constrain when certain types should be used.

Because unary sequences refer to a specific sequence, they may be used to impute a derived sequence. In contrast, the non-unary and uncountable expressions generate a family of sequences.

## Transcript Coordinates

The biological consequences of variants are often inferred from their impacts on transcripts and translated protein sequences. For VRS, we seek to be able to precisely communicate variation in terms of transcript coordinates.

"Transcript" is not well-defined in the community: It may mean vaguely refer to a sequence with an implied biological role, a particular exon structure on a sequence, or, a family of exon structures on different sequences (eg a RefSeq and its projection onto a chromosomal sequence). It is essential to establish a precise definition of Transcript for use in VRS.

The proposed VRS model is that **a Transcript is a set of ordered exon features on a specific sequence, optionally annotated with coding sequence start and end**. A "founding" Transcript may be related to others through projection (alignment) onto other sequences, in which case they form a family of Transcripts based on the founder.

Modeling Transcripts explicitly (as opposed to with generic features) is justified because of the unique and central role that Transcripts play defining a coordinate system to be used when describing variation.

A TranscriptLocation is a location on a Transcript coordinate system, using an interval on the Transcript. (This is analagous to SequenceLocations, Sequences, and sequence intervals.)

### Transcripts

For example, [NM_000314.6](https://www.ncbi.nlm.nih.gov/nuccore/NM_000314.8) is *defined* by RefSeq as having a *transcript sequence* with a structure of nine exons and a CDS region (see below). This Transcript is the founding Transcript. The NCBI [PTEN gene page] (https://www.ncbi.nlm.nih.gov/gene/5728) shows that this (founding) transcript may be projected onto four genomic references: GRCh 37 chr 10, GRCh 38 chr 10, GRCh 38 patch, and a RefSeqGene sequence. The founding transcript and its alignment onto the genomic references constitute a family of five distinct VRS Transcripts.

In [ ]:
# Example transcript: NM_000314.6 (PTEN) on GRCh38 chr 10
# Coordinates written as c. or n. are base-counted, origin 1.
# Other coordinates are interbase.

# NM_000314.6(PTEN):c.78C>A  ~  NC_000010.11:g.87864547C>T

# NM_000314.6 structure on NM_000314.6 sequence
# These exons and CDS on the NM_000314.6 sequence constitute the
# definition of this transcript
t_exons = [(0,1110), (1110,1195), (1195,1240),
            (1240,1284), (1284,1523), (1523,1665),
            (1665,1832), (1832,2057), (2057,8701)]
t_cds = (1031,2243)

# NM_000314.6 aligned (by NCBI) to NC_000010.11 sequence (GRCh38 chr 10), + strand
g_exons = [(87863437, 87864548), (87894024, 87894109), (87925512, 87925557),
           (87931045, 87931089), (87933012, 87933251), (87952117, 87952259),
           (87957852, 87958019), (87960893, 87961118), (87965286, 87971930)]

# Cigars of alignment (relative to transcript)
tg_cigars = "666=1I39=1X404= 85= 45= 44= 239= 142= 167= 225= 6644="

# g_cds is computed from t_cds, accounting for alignment 
# 1032 = 1031 + 1I in cigar
g_cds = (87863437 + 1032, 87965286 + 2243 - 2057)

t_sequence_id = translate_sequence_identifier("refseq:NM_000314.6")
g_sequence_id = translate_sequence_identifier("refseq:NC_000010.11")

In [ ]:
t_transcript = models.Transcript(
  sequence_id = t_sequence_id,
  exons = [models.SimpleInterval(start=ex[0], end=ex[1]) for ex in t_exons],
  cds = models.SimpleInterval(start=t_cds[0], end=t_cds[1])
)
t_transcript.as_dict()

In [ ]:
t_transcript_id = ga4gh_identify(t_transcript)
t_transcript_id

In [ ]:
g_transcript = models.Transcript(
  sequence_id = g_sequence_id,
  exons = [models.SimpleInterval(start=ex[0], end=ex[1]) for ex in g_exons],
  cds = models.SimpleInterval(start=g_cds[0], end=g_cds[1])
)
g_transcript.as_dict()

In [ ]:
g_transcript_id = ga4gh_identify(g_transcript)
g_transcript_id

### TranscriptLocations and TranscriptIntervals

Example: NM_000314.6:c.78C>A from [RCV000204337.5](https://www.ncbi.nlm.nih.gov/clinvar/48116316/).

The CDS start of RefSeq transcript NM_000314.6 is 1032; that is, c.1 corresponds to n.1032 (+1031). Therefore, c.78 (interbase <77,78>)corresponds to n.1109, or interbase interval <1108,1109>.

**☛ VRS does not support CDS numbering explicitly, but does provide the information required to compute it easily.** 

In [ ]:
t_tloc = models.TranscriptLocation(
    transcript = t_transcript,
    interval = models.TranscriptInterval(
        start=models.BaseOffset(base=1108),
        end=models.BaseOffset(base=1109)))
t_tloc.as_dict()

In [2]:
# or, more concisely with the identified transcript:
vrs_enref(t_tloc).as_dict()

NameError: name 'vrs_enref' is not defined

In [ ]:
# A CDS coordinate must be computed using the transcript information, as follows:
t_tloc.interval.start.base - t_transcript.cds.start + 1

In [ ]:
# NM_000314.6:c.78C>A corresponds to NC_000010.11:g.87864547C>T.
# NC_000010.11 contains a 1 nt insertion at interbase 666 relative to NM_000314.6.
# Therefore, <1108,1109> on NM_000314.6 corresponds to <1109,1110> on NC_000010.11.

g_tloc = models.TranscriptLocation(
    transcript = g_transcript,
    interval = models.TranscriptInterval(
        start=models.BaseOffset(base=1109),
        end=models.BaseOffset(base=1110)))
vrs_enref(g_tloc).as_dict()

In [ ]:
# "offset positions"
# NM_000314.6(PTEN):c.79+3A>G ~ NC_000010.11:g.87864551A>G

### Transcript Feature Locations

In [ ]:
tf = models.TranscriptFeature(feature_type="exon", index=0)
tf.as_dict()

In [ ]:
tfi = models.TranscriptFeatureInterval(
    start=models.TranscriptFeature(feature_type="exon", index=0),
    end=models.TranscriptFeature(feature_type="exon", index=5),
)
tfi.as_dict()

In [ ]:
tfl = models.TranscriptLocation(
    transcript = g_transcript,
    interval = tfi)
tfl.as_dict()

In [ ]:
ga4gh_identify(tfl)

In [ ]:
vrs_enref(tfl).as_dict()